<a href="https://colab.research.google.com/github/ScriptSherpa/Building-Code-Agents-with-Hugging-Face-smolagents/blob/main/Lesson2_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <tr>
 <td><img src="icecreamtruck.jpeg" alt="Ice Cream Truck" width="150"/></td>
 <td align="left"><h1>Lesson 2: Introduction to Code Agents</h1></td>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code>  file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [2]:
!pip install python-dotenv==1.0.0

In [10]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

import os
import io
import IPython.display
from PIL import Image
import base64

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv() # read local .env file

from huggingface_hub import login

login(os.environ['MY_key'])

In [9]:
from dotenv import load_dotenv
import os

# Load environment variables from my.env
load_dotenv('my.env')

# Access the API key
api_key = os.getenv('MY_key')

# Login to Hugging Face Hub
from huggingface_hub import login
login(token=api_key)

In [6]:
!ls -a

.  ..  .config	.env  sample_data


In [7]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(dotenv_path=os.path.join(os.getcwd(), 'my.env'))

In [11]:
import pandas as pd

suppliers_data = {
    "name": [
        "Montreal Ice Cream Co",
        "Brain Freeze Brothers",
        "Toronto Gelato Ltd",
        "Buffalo Scoops",
        "Vermont Creamery",
    ],
    "location": [
        "Montreal, QC",
        "Burlington, VT",
        "Toronto, ON",
        "Buffalo, NY",
        "Portland, ME",
    ],
    "distance_km": [120, 85, 400, 220, 280],
    "canadian": [True, False, True, False, False],
    "price_per_liter": [1.95, 1.91, 1.82, 2.43, 2.33],
    "tasting_fee": [0, 12.50, 30.14, 42.00, 0.20],
}

data_description = """Suppliers have an additional tasting fee: that is a fixed fee applied to each order to taste the ice cream."""
suppliers_df = pd.DataFrame(suppliers_data)
suppliers_df

,name,location,distance_km,canadian,price_per_liter,tasting_fee
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20


## Setup tools

In [12]:
import numpy as np

def calculate_daily_supplier_price(row):
    order_volume = 30

    # Calculate raw product cost
    product_cost = row["price_per_liter"] * order_volume

    # Calculate transport cost
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    transport_cost = row["distance_km"] * cost_per_km * trucks_needed

    # Calculate tariffs for ice cream imported from Canada
    tariff = product_cost * np.pi / 50 * row["canadian"]

    # Get total cost
    total_cost = product_cost + transport_cost + tariff + row["tasting_fee"]
    return total_cost

suppliers_df["daily_price"] = suppliers_df.apply(calculate_daily_supplier_price, axis=1)
display(suppliers_df)

# Let's remove this column now.
suppliers_df = suppliers_df.drop("daily_price", axis=1)

,name,location,distance_km,canadian,price_per_liter,tasting_fee,daily_price
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00,206.175663
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50,171.800000
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14,568.170619
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00,378.900000
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20,406.100000


In [14]:
!pip install smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.2 MB/s eta 0:00:00


In [15]:
# First we make a few tools
from smolagents import tool

@tool
def calculate_transport_cost(distance_km: float, order_volume: float) -> float:
    """
    Calculate transportation cost based on distance and order size.
    Refrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.

    Args:
        distance_km: the distance in kilometers
        order_volume: the order volume in liters
    """
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    return distance_km * cost_per_km * trucks_needed


@tool
def calculate_tariff(base_cost: float, is_canadian: bool) -> float:
    """
    Calculates tariff for Canadian imports. Returns the tariff only, not the total cost.
    Assumes tariff on dairy products from Canada is worth 2 * pi / 100, approx 6.2%

    Args:
        base_cost: the base cost of goods, not including transportation cost.
        is_canadian: wether the import is from Canada.
    """
    if is_canadian:
        return base_cost * np.pi / 50
    return 0

In [ ]:
calculate_transport_cost.description

## Setup the Model

In [20]:
%%writefile helper.py

def get_huggingface_token():
  """Returns the Hugging Face API token from the environment variable 'HF_API_KEY'.

  Raises:
    KeyError: If the 'HF_API_KEY' environment variable is not set.
  """
  import os
  return os.environ['MY_key']

Overwriting helper.py


In [21]:
from helper import get_huggingface_token

# Get the token
token = get_huggingface_token()

# Use the token in your code
# ...

ImportError: cannot import name 'get_huggingface_token' from 'helper' (unknown location)

In [22]:
!ls -a

.  ..  .config	.env  helper.py  sample_data


In [23]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "helper.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)
token = helper.get_huggingface_token()

In [25]:
%%writefile helper.py

def get_huggingface_token():
  """Returns the Hugging Face API token from the environment variable 'HF_API_KEY'.

  Raises:
    KeyError: If the 'HF_API_KEY' environment variable is not set.
  """
  import os
  return os.environ['MY_key']
if __name__ == "__main__":
  pass

Overwriting helper.py


In [26]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "helper.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)
token = helper.get_huggingface_token()

In [28]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "helper.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)
token = helper.get_huggingface_token()

In [29]:
from smolagents import HfApiModel, CodeAgent
#from helper import get_huggingface_token

model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    provider="together", # Choose a specific inference provider
    max_tokens=4096,
    temperature=0.1,
    token=token # <-- Pass the retrieved token here
)

## Setup the Code Agent

In [30]:
agent = CodeAgent(
    model=model,
    tools=[calculate_transport_cost, calculate_tariff],
    max_steps=10,
    additional_authorized_imports=["pandas", "numpy"],
    verbosity_level=2
)
agent.logger.console.width=66

In [31]:
agent.run(
    """Can you get me the transportation cost for 50 liters
    of ice cream over 10 kilometers?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the transportation cost for 50 liters           │
│     of ice cream over 10 kilometers?                           │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will use the `calculate_transport_cost` function to    
calculate the cost of transporting 50 liters of ice cream over 10 
kilometers.                                                       
Code:                                                             
```py                                                             
distance_km = 10                                                  
order_volume = 50                                                 
transport_cost = calculate_transport_cost(distance_km,            
order_volume)                                                     
print(f"The transportation cost for 50 liters of ice cream over 10
kilometers is ${transport_cost:.2f}.")                            
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  distance_km = 10                                                
  order_volume = 50                                               
  transport_cost = calculate_transport_cost(distance_km,          
  order_volume)                                                   
  print(f"The transportation cost for 50 liters of ice cream      
  over 10 kilometers is ${transport_cost:.2f}.")                  
 ────────────────────────────────────────────────────────────────

Execution logs:
The transportation cost for 50 liters of ice cream over 10 
kilometers is $12.00.

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: I have the transportation cost for 50 liters of ice cream
over 10 kilometers. I will now provide the final answer.          
Code:                                                             
```py                                                             
final_answer(12.00)                                               
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(12.00)                                             
 ────────────────────────────────────────────────────────────────

Out - Final answer: 12.0

[Step 2: Duration 1.25 seconds| Input tokens: 4,608 | Output 
tokens: 142]

12.0

## Give the agent detailed instructions

In [32]:
task = """Here is a dataframe of different ice cream suppliers.
Could you give me a comparative table (as a dataframe) of the total
daily price for getting daily ice cream delivery from each of them,
given that we need exactly 30 liters of ice cream per day? Take
into account transportation cost and tariffs.
"""
agent.logger.level = 1 # Lower verbosity level
agent.run(
    task,
    additional_args={"suppliers_data": suppliers_df, "data_description": data_description},
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Here is a dataframe of different ice cream suppliers.          │
│ Could you give me a comparative table (as a dataframe) of the  │
│ total                                                          │
│ daily price for getting daily ice cream delivery from each of  │
│ them,                                                          │
│ given that we need exactly 30 liters of ice cream per day?     │
│ Take                                                           │
│ into account transportation cost and tariffs.                  │
│                                                                │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
502 Server Error: Bad Gateway for url: 
https://router.huggingface.co/together/v1/chat/completions

[Step 1: Duration 4.26 seconds| Input tokens: 2,419 | Output 
tokens: 48]

AgentGenerationError: Error in generating model output:
502 Server Error: Bad Gateway for url: https://router.huggingface.co/together/v1/chat/completions

In [35]:
task = """Here is a dataframe of different ice cream suppliers.
Could you give me a comparative table (as a dataframe) of the total
daily price for getting daily ice cream delivery from each of them,
given that we need exactly 30 liters of ice cream per day? Take
into account transportation cost and tariffs.
"""
agent.logger.level = 1 # Lower verbosity level
agent.run(
    task,
    additional_args={"suppliers_data": suppliers_df, "data_description": data_description},
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Here is a dataframe of different ice cream suppliers.          │
│ Could you give me a comparative table (as a dataframe) of the  │
│ total                                                          │
│ daily price for getting daily ice cream delivery from each of  │
│ them,                                                          │
│ given that we need exactly 30 liters of ice cream per day?     │
│ Take                                                           │
│ into account transportation cost and tariffs.                  │
│                                                                │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────────────────────────────╮
│ Calling tool: 'calculate_transport_cost' with arguments:       │
│ {'distance_km': 120, 'order_volume': 30}                       │
╰────────────────────────────────────────────────────────────────╯

Observations: 144.0

[Step 1: Duration 0.06 seconds| Input tokens: 1,742 | Output 
tokens: 23]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 11 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Here  │
│ is the comparative table of the total daily price for getting  │
│ daily ice cream delivery from each supplier, given that we     │
│ need exactly 30 liters of ice cream per day, taking into       │
│ account transportation cost and tariffs: \\\\n\\\\n| Supplier  │
│ Name | Total Daily Price | \\\\n| --- | --- | \\\\n| Montreal  │
│ Ice Cream Co | 155.05 | \\\\n| Brain Freeze Brothers | 193.50  │
│ | \\\\n| Toronto Gelato Ltd | 510.14 | \\\\n| Buffalo Scoops | │
│ 327.90 | \\\\n| Vermont Creamery | 347.20 | '}                 │
╰────────────────────────────────────────────────────────────────╯

Final answer: Here is the comparative table of the total daily 
price for getting daily ice cream delivery from each supplier, 
given that we need exactly 30 liters of ice cream per day, taking 
into account transportation cost and tariffs: \\n\\n| Supplier 
Name | Total Daily Price | \\n| --- | --- | \\n| Montreal Ice 
Cream Co | 155.05 | \\n| Brain Freeze Brothers | 193.50 | \\n| 
Toronto Gelato Ltd | 510.14 | \\n| Buffalo Scoops | 327.90 | \\n| 
Vermont Creamery | 347.20 | 

[Step 11: Duration 0.06 seconds| Input tokens: 24,390 | Output 
tokens: 447]

'Here is the comparative table of the total daily price for getting daily ice cream delivery from each supplier, given that we need exactly 30 liters of ice cream per day, taking into account transportation cost and tariffs: \\\\n\\\\n| Supplier Name | Total Daily Price | \\\\n| --- | --- | \\\\n| Montreal Ice Cream Co | 155.05 | \\\\n| Brain Freeze Brothers | 193.50 | \\\\n| Toronto Gelato Ltd | 510.14 | \\\\n| Buffalo Scoops | 327.90 | \\\\n| Vermont Creamery | 347.20 | '

## Compare to traditional tool calling.

In [33]:
from smolagents import ToolCallingAgent

model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    temperature=0.6
)
agent = ToolCallingAgent(
    model=model,
    tools=[calculate_transport_cost, calculate_tariff],
    max_steps=20,
)
agent.logger.console.width=66

<div style="font-size: 14px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> During this run, the agent may generate code that creates an error. This is OK. The agent will try to fix the error.
</div>


In [34]:
output = agent.run(
    task,
    additional_args={"suppliers_data": suppliers_df, "data_description": data_description},
)
print(output)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Here is a dataframe of different ice cream suppliers.          │
│ Could you give me a comparative table (as a dataframe) of the  │
│ total                                                          │
│ daily price for getting daily ice cream delivery from each of  │
│ them,                                                          │
│ given that we need exactly 30 liters of ice cream per day?     │
│ Take                                                           │
│ into account transportation cost and tariffs.                  │
│                                                                │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────────────────────────────╮
│ Calling tool: 'calculate_transport_cost' with arguments:       │
│ {'distance_km': 120, 'order_volume': 30}                       │
╰────────────────────────────────────────────────────────────────╯

Observations: 144.0

[Step 1: Duration 1.31 seconds| Input tokens: 1,742 | Output 
tokens: 23]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 11 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Here  │
│ is the comparative table of the total daily price for getting  │
│ daily ice cream delivery from each supplier, given that we     │
│ need exactly 30 liters of ice cream per day, taking into       │
│ account transportation cost and tariffs: \\\\n\\\\n| Supplier  │
│ Name | Total Daily Price | \\\\n| --- | --- | \\\\n| Montreal  │
│ Ice Cream Co | 155.05 | \\\\n| Brain Freeze Brothers | 193.50  │
│ | \\\\n| Toronto Gelato Ltd | 510.14 | \\\\n| Buffalo Scoops | │
│ 327.90 | \\\\n| Vermont Creamery | 347.20 | '}                 │
╰────────────────────────────────────────────────────────────────╯

Final answer: Here is the comparative table of the total daily 
price for getting daily ice cream delivery from each supplier, 
given that we need exactly 30 liters of ice cream per day, taking 
into account transportation cost and tariffs: \\n\\n| Supplier 
Name | Total Daily Price | \\n| --- | --- | \\n| Montreal Ice 
Cream Co | 155.05 | \\n| Brain Freeze Brothers | 193.50 | \\n| 
Toronto Gelato Ltd | 510.14 | \\n| Buffalo Scoops | 327.90 | \\n| 
Vermont Creamery | 347.20 | 

[Step 11: Duration 7.87 seconds| Input tokens: 24,390 | Output 
tokens: 447]

Here is the comparative table of the total daily price for getting daily ice cream delivery from each supplier, given that we need exactly 30 liters of ice cream per day, taking into account transportation cost and tariffs: \\n\\n| Supplier Name | Total Daily Price | \\n| --- | --- | \\n| Montreal Ice Cream Co | 155.05 | \\n| Brain Freeze Brothers | 193.50 | \\n| Toronto Gelato Ltd | 510.14 | \\n| Buffalo Scoops | 327.90 | \\n| Vermont Creamery | 347.20 | 
